<a href="https://colab.research.google.com/github/ThierrryScotto/Finetunig-BERT/blob/main/Finetunig_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
!pip install -U -q transformers
!pip install -U -q accelerate
!pip install -U -q datasets
!pip install -U -q bertviz
!pip install -U -q umap-learn
!pip install -U -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 8.5 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoModel

In [ ]:
url_dataset = 'https://raw.githubusercontent.com/ThierrryScotto/Finetunig-BERT/main/twitter_multi_class_sentiment.csv'
df = pd.read_csv(url_dataset)
df

# Data Analysis

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['label'].value_counts()

In [ ]:
label_counts = df['label_name'].value_counts(ascending=True)
label_counts.plot.barh()
plt.title('Frequency of Classes')
plt.ylabel('Emotions')

In [ ]:
df['words per tweet'] = df['text'].str.split().apply(len)
df

In [ ]:
df.boxplot("words per tweet", by='label_name')

# Data Preparation

### Tokenizing Test

In [ ]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
text = "I love machne learning! Tokenization is awesome!"

encoded_text = tokenizer(text)
print(encoded_text)

In [ ]:
tokenizer.vocab

In [ ]:
len(tokenizer.vocab)

## Data Loader and Train Test Split

In [ ]:
df

In [ ]:
train, test = train_test_split(df, test_size=0.3, stratify=df['label_name'])
test, validation = train_test_split(test, test_size=1/3, stratify=test['label_name'])

In [ ]:
print(train.shape)
print(test.shape)
print(validation.shape)

In [ ]:
dataset = DatasetDict(
    {
        'train': Dataset.from_pandas(train, preserve_index=False),
        'test': Dataset.from_pandas(test, preserve_index=False),
        'validation': Dataset.from_pandas(validation, preserve_index=False)
    }
)

dataset

## Tokenization of the emotions

In [ ]:
dataset['train'][129]

In [ ]:
def tokenize(batch):
    temp = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    return temp

In [ ]:
print(tokenize(dataset['train'][129]))

In [ ]:
print(tokenize(dataset['train'][:2]))

In [ ]:
dataset['train'][0], dataset['train'][1]

dataset with tokenized texts

In [ ]:
emotion_encoded = dataset.map(tokenize, batched=True, batch_size=None)

In [ ]:
emotion_encoded['train'][0]

In [ ]:
# label2id and id2label

label2id = { x['label_name'] : x['label'] for x in dataset['train'] }
id2label = { v : k for k, v in label2id.items() }

label2id, id2label

# Model Building

In [ ]:
model= AutoModel.from_pretrained(model_ckpt)

In [ ]:
model

In [ ]:
model.config.architectures

In [ ]:
model.config

## Fine-Tuning

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig

In [ ]:
# number of emotions labels
num_labels = len(label2id)

# try to use GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fine-tunig
config = AutoConfig.from_pretrained(model_ckpt, num_labels=num_labels, label2id=label2id, id2label=id2label)

# Applying fine-tuning and saving it in GPU or CPU
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)

In [ ]:
device

In [ ]:
model

## Compute metrics

In [ ]:
import evaluate

accuracy = evaluate.load('accuracy')

In [ ]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

## Training

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
batch_size = 64

training_dir = "bert_base_train_dir"

training_args = TrainingArguments(
    output_dir = training_dir,
    overwrite_output_dir = True,
    num_train_epochs = 2,
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    disable_tqdm = False
)

In [ ]:
tokenizer

In [ ]:
trainer = Trainer(model = model,
                  args = training_args,
                  compute_metrics = compute_metrics,
                  train_dataset = emotion_encoded['train'],
                  eval_dataset = emotion_encoded['validation'],
                  tokenizer = tokenizer)

In [ ]:
trainer.train()